In [1]:
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, ModelMetadataServiceApi, DriftDetectorApi, BatchJobsApi, BatchJobDefinition
from seldon_deploy_sdk.auth import OIDCAuthenticator
from seldon_deploy_sdk.rest import ApiException
import pprint
import json

In [2]:
SD_IP = "34.74.242.235"
config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = 'client_credentials'
config.verify_ssl = False
auth = OIDCAuthenticator(config)
config.id_token = auth.authenticate()
seldon_api_client = ApiClient(configuration=config, authenticator=auth)

In [3]:
YOUR_NAME = "josh-preview-2"
MODEL_NAME = "lr"

DEPLOYMENT_NAME = f"{YOUR_NAME}-{MODEL_NAME}"
MODEL_LOCATION = f"gs://tom-seldon-examples/retail-workshop/models/josh/lr"

In [4]:
NAMESPACE = "seldon-demos"

PREPACKAGED_SERVER = "SKLEARN_SERVER"

CPU_REQUESTS = "0.1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "0.1"
MEMORY_LIMITS = "1Gi"

mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "transport": "rest",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "implementation": PREPACKAGED_SERVER,
                    "modelUri": MODEL_LOCATION,
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}

In [5]:
deployment_api = SeldonDeploymentsApi(seldon_api_client)
deployment_api.create_seldon_deployment(namespace="seldon-demos", mldeployment=mldeployment)

{'api_version': 'machinelearning.seldon.io/v1alpha2',
 'kind': 'SeldonDeployment',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': None,
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': {'fluentd': 'true'},
              'managed_fields': None,
              'name': 'josh-preview-2-lr',
              'namespace': 'seldon-demos',
              'owner_references': None,
              'resource_version': None,
              'self_link': None,
              'uid': None},
 'spec': {'annotations': {'seldon.io/engine-seldon-log-messages-externally': 'true'},
          'name': 'josh-preview-2-lr',
          'oauth_key': None,
          'oauth_secret': None,
          'predictors': [{'annotations': None,
                          'component_specs': [{'hpa_spec': No

## Check Status of Deployment

In [16]:
deployment_status = ''
while deployment_status != "Available":
    try:
        api_response = deployment_api.read_seldon_deployment(DEPLOYMENT_NAME, NAMESPACE)
        pprint.pprint(api_response.status.state.replace('\'', ''))
        deployment_status = api_response.status.state
    except ApiException as e:
        print("Exception when calling SeldonDeploymentsApi->read_seldon_deployment: %s\n" % e)

'Available'


In [17]:
api_response = deployment_api.read_seldon_deployment(DEPLOYMENT_NAME, NAMESPACE)
pprint.pprint(api_response)

{'api_version': 'machinelearning.seldon.io/v1',
 'kind': 'SeldonDeployment',
 'metadata': {'annotations': {'kubectl.kubernetes.io/last-applied-configuration': '{"apiVersion":"machinelearning.seldon.io/v1alpha2","kind":"SeldonDeployment","metadata":{"annotations":{},"creationTimestamp":null,"labels":{"argocd.argoproj.io/instance":"seldon-gitops-seldon-demos","fluentd":"true"},"name":"josh-preview-2-lr","namespace":"seldon-demos"},"spec":{"annotations":{"seldon.io/engine-seldon-log-messages-externally":"true"},"name":"josh-preview-2-lr","predictors":[{"componentSpecs":[{"metadata":{"creationTimestamp":"2022-07-21T21:06:40Z"},"spec":{"containers":[{"name":"josh-preview-2-lr-container","resources":{"limits":{"cpu":"100m","memory":"1Gi"},"requests":{"cpu":"100m","memory":"1Gi"}}}]}}],"engineResources":{},"graph":{"endpoint":{"type":"REST"},"implementation":"SKLEARN_SERVER","logger":{"mode":"all"},"modelUri":"gs://tom-seldon-examples/retail-workshop/models/josh/lr","name":"josh-preview-2-lr-

## Get list of Deployments Via Namespace

In [34]:
deployment_api = SeldonDeploymentsApi(seldon_api_client)

try:
    api_response = deployment_api.list_seldon_deployments(NAMESPACE)
    pprint.pprint(api_response)
except ApiException as e:
    print("Exception when calling SeldonDeploymentsApi->list_seldon_deployments: %s\n" % e)

{'api_version': None,
 'items': [{'api_version': 'machinelearning.seldon.io/v1',
            'kind': 'SeldonDeployment',
            'metadata': {'annotations': {'kubectl.kubernetes.io/last-applied-configuration': '{"apiVersion":"machinelearning.seldon.io/v1alpha2","kind":"SeldonDeployment","metadata":{"annotations":{},"creationTimestamp":null,"labels":{"argocd.argoproj.io/instance":"seldon-gitops-seldon-demos","fluentd":"true"},"name":"josh-lr","namespace":"seldon-demos"},"spec":{"annotations":{"seldon.io/engine-seldon-log-messages-externally":"true"},"name":"josh-lr","predictors":[{"componentSpecs":[{"metadata":{"creationTimestamp":"2022-07-13T13:30:36Z"},"spec":{"containers":[{"name":"josh-lr-container","resources":{"limits":{"cpu":"100m","memory":"1Gi"},"requests":{"cpu":"100m","memory":"1Gi"}}}]}}],"engineResources":{},"graph":{"endpoint":{"type":"REST"},"implementation":"SKLEARN_SERVER","logger":{"mode":"all"},"modelUri":"gs://tom-seldon-examples/retail-workshop/models/josh/lr","